In [792]:
import math
import time
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import path
from matplotlib.path import Path
import skimage.transform as sktr
import skimage.color as color
import skimage.io as skio
import scipy.misc
from scipy import interpolate
from scipy.spatial import Delaunay
import scipy
from pylab import plot, ginput, show, axis
plt.rcParams['image.cmap'] = 'gray'
# %pylab inline
%pylab qt

Populating the interactive namespace from numpy and matplotlib


# Defining Correspondences

### Functions

In [728]:
def getpoints(img):
    plt.figure()
    plt.imshow(img)
    pts = ginput(33, timeout=500)
    pts = [(0,0),(0,img.shape[0]-1),(img.shape[1]-1,0),(img.shape[1]-1,img.shape[0]-1)]+pts
    pts = np.array(pts)
    plt.close()
    plt.figure()
    plt.imshow(img)
    plt.plot(pts[:,0], pts[:,1], "r+")
    plt.show()
    return pts

In [729]:
def getavg(pts1,pts2):
    pts_avg = pts1+0.5*(pts2-pts1)
    return pts_avg

### Main

In [730]:
#Get points
#img1
img1 = plt.imread('face1.jpg')/255.
pts1 = getpoints(img1)
# print pts1

#img2
img2 = plt.imread('face2.jpg')/255.
pts2 = getpoints(img2)
# print pts2

In [731]:
#Get average
pts_avg = getavg(pts1,pts2)
plt.figure()
plt.imshow(img1)
plt.plot(pts_avg[:,0], pts_avg[:,1], "r+")
plt.figure()
plt.imshow(img2)
plt.plot(pts_avg[:,0], pts_avg[:,1], "r+")
plt.show()

In [732]:
#Compute Delaunay triangulation
triangles = Delaunay(pts_avg)
triPoints = triangles.simplices.copy()
plt.figure()
plt.imshow(img1)
plt.triplot(pts1[:,0], pts1[:,1], triPoints)
plt.plot(pts1[:,0], pts1[:,1], 'r+')
plt.figure()
plt.imshow(img2)
plt.triplot(pts2[:,0], pts2[:,1], triPoints)
plt.plot(pts2[:,0], pts2[:,1], 'r+')
plt.show()

# Computing the "Mid-Way Face"

### Functions

In [733]:
def computeAffine(tri1_pts, tri2_pts):
    A = np.array([tri1_pts[:,0],tri1_pts[:,1],[1,1,1]])
    X = np.array([tri2_pts[:,0],tri2_pts[:,1],[1,1,1]])
    Ainv = numpy.linalg.inv(A)
    T = np.dot(X,Ainv)
    Tinv = numpy.linalg.inv(T)
    return Tinv

In [734]:
def interpolate(img, x, y):
    x0 = int(np.floor(x))
    x1 = x0 + 1
    y0 = int(np.floor(y))
    y1 = y0 + 1

    x0 = np.clip(x0, 0, img.shape[1]-1);
    x1 = np.clip(x1, 0, img.shape[1]-1);
    y0 = np.clip(y0, 0, img.shape[0]-1);
    y1 = np.clip(y1, 0, img.shape[0]-1);

    imga = img[ y0, x0 ]
    imgb = img[ y1, x0 ]
    imgc = img[ y0, x1 ]
    imgd = img[ y1, x1 ]

    wa = (x1-x) * (y1-y)
    wb = (x1-x) * (y-y0)
    wc = (x-x0) * (y1-y)
    wd = (x-x0) * (y-y0)

    return wa*imga + wb*imgb + wc*imgc + wd*imgd

In [793]:
def inversewarp(img, nimg, A_set, triPoints, avg_shape, triPaths, interp):
    points = [[x,y] for x in range(nimg.shape[1]) for y in range(nimg.shape[0])]

    triLabels = {}
    for point in points:
        triLabels[str(point)] = 0 
        
    for j, t in enumerate(triPaths):
        minx = int(math.floor(min(t.vertices[:,0])))
        maxx = int(math.ceil(max(t.vertices[:,0])))
        miny = int(math.floor(min(t.vertices[:,1])))
        maxy = int(math.ceil(max(t.vertices[:,1])))
        minipoints = [[x,y] for x in range(minx,maxx+1) for y in range(miny,maxy+1)]
        locs = t.contains_points(minipoints, transform=None)
        locs = np.dot(locs.astype(int),1)
        locs = [minipoints[i] for i, x in enumerate(locs) if x==1]
        for loc in locs:
            triLabels[str(loc)] = j
    
    for i, pt in enumerate(points):
        Tinv = A_set[:,:,triLabels[str(pt)]]
        point = np.array([[pt[0]],[pt[1]],[1]])
        ndot = np.dot(Tinv,point)
        if interp:
            nimg[pt[1],pt[0],:] = interpolate(img,np.array(ndot[0][0]),np.array(ndot[1][0]))
        else:
            npoint = (min(img.shape[1]-1,max(0,ndot[0][0])),min(img.shape[0]-1,max(0,ndot[1][0])))
            nimg[pt[1],pt[0],:] = img[int(npoint[1]),int(npoint[0]),:]
    return nimg

In [790]:
def computemid(img, img_pts, avg_shape, triPoints, interp):
    nimg = numpy.zeros(img.shape)
    t = triPoints
    A_set = numpy.zeros((3,3,t.shape[0]))
    tpoints = np.array([[avg_shape[x[0]],avg_shape[x[1]],avg_shape[x[2]]] for x in triPoints])
    triPaths = []
    
    for x in range(0,t.shape[0]):
        tri1_pts = np.array([img_pts[t[x][0]],img_pts[t[x][1]],img_pts[t[x][2]]])
        tri2_pts = np.array([avg_shape[t[x][0]],avg_shape[t[x][1]],avg_shape[t[x][2]]])
        A = computeAffine(tri1_pts, tri2_pts)
        A_set[:,:,x] = A
        
        tpts = tpoints[x]
        triPaths.append(Path(tpts))
        
    inversewarp(img, nimg, A_set, triPoints, avg_shape, triPaths, interp)
    return nimg

### Main

In [747]:
#Compute average shape "Use from part 1"
avg_shape = pts_avg

In [794]:
#Warp both faces into that shape "Affine warp"
# If you want to use interpolation change False to True, however it is 25 seconds slower
#then without interpolation.
nimg1 = computemid(img1,pts1,avg_shape, triPoints, False)
nimg2 = computemid(img2,pts2,avg_shape, triPoints, False)

In [795]:
#Show and Save Warped Images
plt.figure()
plt.imshow(nimg1)
filename = "facewarped1.jpg"
scipy.misc.toimage(nimg1).save(filename)
plt.figure()
plt.imshow(nimg2)
filename = "facewarped2.jpg"
scipy.misc.toimage(nimg2).save(filename)
plt.show()

In [750]:
#Average colors together
img_new = nimg1+0.5*(nimg2-nimg1)
plt.figure()
plt.imshow(img_new)
plt.show()
filename = "midwayface2.jpg"
scipy.misc.toimage(img_new).save(filename)

# The Morph Sequence

### Functions

In [752]:
def morph(im1, im2, im1_pts, im2_pts, tri, warp_frac, dissolve_frac):
    avg_shape = im1_pts+warp_frac*(im2_pts-im1_pts)
    nimg1 = computemid(im1,im1_pts,avg_shape, tri, False)
    nimg2 = computemid(im2,im2_pts,avg_shape, tri, False)
    nimg = nimg1+dissolve_frac*(nimg2-nimg1)
    return nimg

### Main

In [753]:
#Morph function
for frac in range(0,46):
    warp_frac = (1./45.)*frac
    dissolve_frac = (1./45.)*frac
    morphed_im = morph(img1, img2, pts1, pts2, triPoints, warp_frac, dissolve_frac)
    filename = "out/out"+str(frac)+".jpg"
    scipy.misc.toimage(morphed_im).save(filename)


# The "Mean Face" Of a Population

### Main

In [756]:
#Get data set

## Images
dataset = np.zeros((480,640,3,40))
datasetf = np.zeros((480,640,3,10))
datasetm = np.zeros((480,640,3,30))
files1 = glob.glob('data/*.bmp')
fcount = 0
mcount = 0
for x in range(len(files1)):
    if "1f" in files1[x]:
        datasetf[:,:,:,fcount] = plt.imread(files1[x])/255.
        fcount += 1
    if "1m" in files1[x]:
        datasetm[:,:,:,mcount] = plt.imread(files1[x])/255.
        mcount += 1
    dataset[:,:,:,x] = plt.imread(files1[x])/255.
    
## Points
files2 = glob.glob('data/*.asf')
full_points = []
full_pointsf = []
full_pointsm = []
for x in range(len(files2)):
    fi = open(files2[x])
    next = fi.readline()
    count = 0
    points = []
    while next != "":
        if count >= 16 and count <= 73:
            nt = next.split()
            points += [[float(nt[2])*640.,float(nt[3])*480.]]

        next = fi.readline()
        count += 1
    points = [(0,0),(0,480-1),(640-1,0),(640-1,480-1)]+points
    if "1f" in files2[x]:
        full_pointsf += [points]
    if "1m" in files2[x]:
        full_pointsm += [points]
    full_points += [points]
full_points = np.array(full_points)
full_pointsf = np.array(full_pointsf)
full_pointsm = np.array(full_pointsm)

In [472]:
#My face with their point system
plt.figure()
menew = plt.imread('menew.jpg')/255.
plt.imshow(menew)
ptsnew = ginput(58, timeout=500)
plt.close()
ptsnew = np.array(ptsnew)
ptsnew = np.array([(0,0),(0,480-1),(640-1,0),(640-1,480-1)]+ ptsnew.tolist())

In [757]:
#Compute average face shape
sum_shape = np.copy(full_points[0])
for x in range(1,full_points.shape[0]):
    sum_shape += full_points[x]
data_avg_shape = sum_shape/full_points.shape[0]

In [758]:
#Compute triangles
data_triangles = Delaunay(data_avg_shape)
data_triPoints = data_triangles.simplices.copy()

In [759]:
#Morph each face in the dataset into average shape
morphed_dataset = np.zeros((480,640,3,40))
for x in range(full_points.shape[0]):
    morphed_dataset[:,:,:,x] = computemid(dataset[:,:,:,x],full_points[x],data_avg_shape, 
                                          data_triPoints, False)

In [760]:
#Compute average face
average_face = np.copy(morphed_dataset[:,:,:,0])
for x in range(1,full_points.shape[0]):
    average_face += morphed_dataset[:,:,:,x]
average_face = average_face/full_points.shape[0]
plt.figure()
plt.imshow(average_face)
plt.show
filename = "avg.jpg"
scipy.misc.toimage(average_face).save(filename)

In [761]:
#My face warped into average
data_my_face = computemid(menew,ptsnew,data_avg_shape, data_triPoints, False)
plt.figure()
plt.imshow(data_my_face)
plt.show
filename = "mwarp.jpg"
scipy.misc.toimage(data_my_face).save(filename)

In [762]:
#Average face warped into my face geometry
avg_face_tome = computemid(average_face,data_avg_shape,ptsnew, data_triPoints, False)
plt.figure()
plt.imshow(avg_face_tome)
plt.show
filename = "awarp.jpg"
scipy.misc.toimage(avg_face_tome).save(filename)

# Caricatures

### Main

In [764]:
#Extrapolate My unique features
caricature_geo = ptsnew +0.5*(ptsnew-data_avg_shape)
#Warp
caricature_face = computemid(menew,ptsnew,caricature_geo, data_triPoints, False)
plt.figure()
plt.imshow(caricature_face)
plt.show
filename = "caricature1.jpg"
scipy.misc.toimage(caricature_face).save(filename)

In [766]:
#Extrapolate Female Danes unique features
caricature_geof = full_points[4] +0.5*(full_points[4]-data_avg_shape)
#Warp
caricature_facef = computemid(morphed_dataset[:,:,:,4],full_points[4],caricature_geof, data_triPoints, False)
plt.figure()
plt.imshow(caricature_facef)
plt.show
filename = "carf.jpg"
scipy.misc.toimage(caricature_facef).save(filename)

In [767]:
#Extrapolate Male Danes unique features
caricature_geom = full_points[0] +0.5*(full_points[0]-data_avg_shape)
#Warp
caricature_facem = computemid(morphed_dataset[:,:,:,0],full_points[0],caricature_geom, data_triPoints, False)
plt.figure()
plt.imshow(caricature_facem)
plt.show
filename = "carm.jpg"
scipy.misc.toimage(caricature_facem).save(filename)

# Change Gender

### Main

In [768]:
#Use extrapolation female
#Shape
sum_shapef = np.copy(full_pointsf[0])
for x in range(1,full_pointsf.shape[0]):
    sum_shapef += full_pointsf[x]
data_avg_shapef = sum_shapef/full_pointsf.shape[0]

#Average Image
morphed_datasetf = np.zeros((480,640,3,10))
for x in range(full_pointsf.shape[0]):
    morphed_datasetf[:,:,:,x] = computemid(datasetf[:,:,:,x],full_pointsf[x],data_avg_shapef, data_triPoints, False)
average_facef = np.copy(morphed_datasetf[:,:,:,0])
for x in range(1,full_pointsf.shape[0]):
    average_facef += morphed_datasetf[:,:,:,x]
average_facef = average_facef/full_pointsf.shape[0]
plt.figure()
plt.imshow(average_facef)
plt.show
filename = "fa.jpg"
scipy.misc.toimage(average_facef).save(filename)

In [772]:
#Use extrapolation male
#Shape
sum_shapem = np.copy(full_pointsm[0])
for x in range(1,full_pointsm.shape[0]):
    sum_shapem += full_pointsm[x]
data_avg_shapem = sum_shapem/full_pointsm.shape[0]

#Average Image
morphed_datasetm = np.zeros((480,640,3,30))
for x in range(full_pointsm.shape[0]):
    morphed_datasetm[:,:,:,x] = computemid(datasetm[:,:,:,x],full_pointsm[x],data_avg_shapem, data_triPoints, False)
average_facem = np.copy(morphed_datasetm[:,:,:,0])
for x in range(1,full_pointsm.shape[0]):
    average_facem += morphed_datasetm[:,:,:,x]
average_facem = average_facem/full_pointsm.shape[0]
plt.figure()
plt.imshow(average_facem)
plt.show
filename = "ma.jpg"
scipy.misc.toimage(average_facem).save(filename)

In [774]:
#warp male and female average faces into average shape
f_face = computemid(average_facef,data_avg_shapef,data_avg_shape,data_triPoints, False)
m_face = computemid(average_facem,data_avg_shapem,data_avg_shape,data_triPoints, False)

In [780]:
#female color difference
female_color = (f_face-m_face)
plt.figure()
plt.imshow(female_color)
plt.show
filename = "fc.jpg"
scipy.misc.toimage(female_color).save(filename)

In [781]:
#male color difference
male_color = (m_face-f_face)
plt.figure()
plt.imshow(male_color)
plt.show
filename = "mc.jpg"
scipy.misc.toimage(male_color).save(filename)

### Male

In [782]:
#Just morphing shape
memale_geo = ptsnew + 1.0*(data_avg_shapem-data_avg_shapef)
memale_face = computemid(menew,ptsnew,memale_geo, data_triPoints, False)
plt.figure()
plt.imshow(memale_face)
plt.show
filename = "s1.jpg"
scipy.misc.toimage(memale_face).save(filename)

In [783]:
#Just the appearance
male_color_warped = computemid(male_color,data_avg_shape,ptsnew,data_triPoints, False)
color_change = (menew+male_color_warped)
color_change = (color_change - color_change.min()) / color_change.ptp()
plt.figure()
plt.imshow(color_change)
plt.show
filename = "c1.jpg"
scipy.misc.toimage(color_change).save(filename)

In [784]:
#Both
male_color_warped = computemid(male_color,data_avg_shape,memale_geo,data_triPoints, False)
both_change = (memale_face+male_color_warped)
both_change = (both_change - both_change.min()) / both_change.ptp()
plt.figure()
plt.imshow(both_change)
plt.show
filename = "b1.jpg"
scipy.misc.toimage(both_change).save(filename)

### Female

In [785]:
#Just morphing shape
mefemale_geo = ptsnew + 1.0*(data_avg_shapef-data_avg_shapem)
mefemale_face = computemid(menew,ptsnew,mefemale_geo, data_triPoints, False)
plt.figure()
plt.imshow(mefemale_face)
plt.show
filename = "s2.jpg"
scipy.misc.toimage(mefemale_face).save(filename)

In [786]:
#Just the appearance
female_color_warped = computemid(female_color,data_avg_shape,ptsnew,data_triPoints, False)
color_change = (menew+female_color_warped)
color_change = (color_change - color_change.min()) / color_change.ptp()
plt.figure()
plt.imshow(color_change)
plt.show
filename = "c2.jpg"
scipy.misc.toimage(color_change).save(filename)

In [787]:
#Both
female_color_warped = computemid(female_color,data_avg_shape,mefemale_geo,data_triPoints, False)
both_change = (mefemale_face+female_color_warped)
both_change = (both_change - both_change.min()) / both_change.ptp()
plt.figure()
plt.imshow(color_change)
plt.show
filename = "b2.jpg"
scipy.misc.toimage(both_change).save(filename)